In [1]:
import numpy as np
import numba
from numba import typed
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [2]:
#-------------------------- parametry ---------------------------------
N = 100 # wymiar siatki
J= 1 # calka wymiany
Beta = 1000 # beta
B = 5 # indukcja magnetyczna
liczba_krokow = 100 # liczba krokow symulacji
G = 0.2 # gestosc spinow


In [3]:
def oblicz_delta_E(Sij, Sn): # funkcja obliczajaca zmiane hamiltonianu wywolana spinem S_ij, Sn jest to suma spinow sasiadujacych ze spinem S_ij uwzgledniana w Hamiltonianie 
        return 2 * Sij * (B + J * Sn)

def nastepny_stan_spinu(siatka, i, j): # funkcja mogaca zmodyfikowac stan spinu

    # suma wszystkich spinow sasiadujacych ze spinem S_ij z uwzglednieniem periodycznych warunkow brzegowych (torus)
    Sn = siatka[(i - 1) % N, j] + siatka[(i + 1) % N, j] + siatka[i, (j - 1) % N] + siatka[i, (j + 1) % N]

    dE = oblicz_delta_E(siatka[i, j], Sn) # licze zmiane energii dla zmienionego spinu

    if dE < 0 or np.random.random() < np.exp(-dE*Beta): # jezeli zmiana energii jest ujemna badz wylosowana liczba z przedzialu [0,1] nie miesci sie w prawdopodobienstwie zmiany stanu spinu 
        siatka[i, j] = -siatka[i, j]  

    return siatka[i,j] 

def aktualizuj_stan_siatki(siatka, N):

    for _ in range(N*N): # petla po liczbie spinow (mozna kilka razy wylosowac ten sam spin)

            i = np.random.randint(N) # losowanie dowolnego spinu S_ij (mozna dwa razy wylosowac ten sam spin)
            j = np.random.randint(N)

            siatka[i,j] = nastepny_stan_spinu(siatka, i, j) # siatka wejsciowa jest caly czas modyfikowana, dynamika asynchroniczna

def symulacja(liczba_krokow):
    # siatka wypelniona losowo wartosciami +-1, wylosowanie 1 z prawd. gestosc, wylosowanie -1 z prawd. 1-gestosc
    siatka = np.random.choice([1, -1], size=[N, N], p=[G, 1-G]) 

    stany_siatki = typed.List()
    stany_siatki.append(siatka.copy())

    for _ in range(liczba_krokow):

        aktualizuj_stan_siatki(siatka, N)

        stany_siatki.append(siatka.copy())

    return stany_siatki

stany_siatki = symulacja(liczba_krokow)

In [4]:
@numba.njit(numba.int64(numba.int64, numba.int64))
def oblicz_delta_E(Sij, Sn): # funkcja obliczajaca zmiane hamiltonianu wywolana spinem S_ij, Sn jest to suma spinow sasiadujacych ze spinem S_ij uwzgledniana w Hamiltonianie 
        return 2 * Sij * (B + J * Sn)

In [5]:
@numba.njit(numba.int32(numba.int32[:,:], numba.int64, numba.int64))
def nastepny_stan_spinu(siatka, i, j): # funkcja mogaca zmodyfikowac stan spinu

    # suma wszystkich spinow sasiadujacych ze spinem S_ij z uwzglednieniem periodycznych warunkow brzegowych (torus)
    Sn = siatka[(i - 1) % N, j] + siatka[(i + 1) % N, j] + siatka[i, (j - 1) % N] + siatka[i, (j + 1) % N]

    dE = oblicz_delta_E(siatka[i, j], Sn) # licze zmiane energii dla zmienionego spinu

    if dE < 0 or np.random.random() < np.exp(-dE*Beta): # jezeli zmiana energii jest ujemna badz wylosowana liczba z przedzialu [0,1] nie miesci sie w prawdopodobienstwie zmiany stanu spinu 
        siatka[i, j] = -siatka[i, j]  

    return siatka[i,j] 

In [6]:
@numba.njit()
def aktualizuj_stan_siatki(siatka, N):

    for _ in range(N*N): # petla po liczbie spinow (mozna kilka razy wylosowac ten sam spin)

            i = np.random.randint(N) # losowanie dowolnego spinu S_ij (mozna dwa razy wylosowac ten sam spin)
            j = np.random.randint(N)

            siatka[i,j] = nastepny_stan_spinu(siatka, i, j) # siatka wejsciowa jest caly czas modyfikowana, dynamika asynchroniczna

In [7]:
def symulacja(liczba_krokow):
    # siatka wypelniona losowo wartosciami +-1, wylosowanie 1 z prawd. gestosc, wylosowanie -1 z prawd. 1-gestosc
    siatka = np.random.choice([1, -1], size=[N, N], p=[G, 1-G]) 

    stany_siatki = typed.List()
    stany_siatki.append(siatka.copy())

    for _ in range(liczba_krokow):

        aktualizuj_stan_siatki(siatka, N)

        stany_siatki.append(siatka.copy())

    return stany_siatki

In [8]:
stany_siatki = symulacja(liczba_krokow)

In [9]:
fig, ax = plt.subplots(figsize = (5, 5))
fig.subplots_adjust(0, 0, 1, 1)
img = ax.imshow(stany_siatki[0])

plt.close()

def frame(i):
    img.set_data(stany_siatki[i])
    return img,

anim = FuncAnimation(fig, frame, frames = len(stany_siatki), interval = 200, blit = True)
HTML(anim.to_jshtml())